# Neural network implementation

In [240]:
import numpy as np
import os

from fault_tolerant_ml.data.mnist import MNist
from fault_tolerant_ml.models import nn
import fault_tolerant_ml.activations.activation as F

%reload_ext autoreload
%autoreload 2

## Read in data

In [241]:
data_dir = "../data"
filepaths = {
    "train": {
        "images": os.path.join(data_dir, "train-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "train-labels-idx1-ubyte.gz")
    },
    "test": {
        "images": os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "labels": os.path.join(data_dir, "t10k-labels-idx1-ubyte.gz")
    }
}
mnist = MNist(filepaths)

In [242]:
mnist

<MNist X_train=(60000, 784), y_train=(60000, 10), X_test=(10000, 784), y_test=(10000, 10)>

In [243]:
n_features, n_classes = mnist.X_train.shape[1], mnist.y_train.shape[1]
print(f"n_features={n_features}, n_classes={n_classes}")

n_features=784, n_classes=10


In [244]:
theta = np.random.randn(n_features, n_classes)

In [296]:
class NeuralNet(nn.Model):
    
    def __init__(self):
        
        # MLP - 1 input layer, 1 hidden layer, 1 output layer
        # self.fc1 = nn.Layer(n_inputs=784, n_outputs=128)
        # self.fc2 = nn.Layer(n_inputs=128, n_outputs=10)
        self.layers = []
        # self.act_fn = F.Sigmoid()
        
    def add(self, layer):
        """Add layer to model
        """
        self.layers.append(layer)
        return self
    
    def forward(self, x):
        
        # z1 = self.fc1(x)
        # a1 = self.act_fn(z1)
        # a1, z1 = self.fc1(x)
        # z2 = self.fc2(a1)
        # y_pred = self.act_fn(z2)
        # y_pred, z2 = self.fc2(a1)
        
        a = x
        for layer in self.layers:
            print(layer)
            layer(a)
            a = layer.y
        
        y_pred = a
        return y_pred
    
    def backward():
        pass
    
#     def backward(self, x, y, a_n):
        
#         y_pred, z2, a1, z1 = a_n
#         # Output layer error
#         delta2 = (y_pred - y)# * self.act_fn.grad(z2)
#         # Gradient of cost function
#         dw2 = np.dot(a1.T, delta2)
#         # Backpropagate the error through the network
#         delta1 = np.dot(delta2, self.fc2.W.T) * self.act_fn.grad(z1)
#         # Calculate gradient
#         dw1 = np.dot(x.T, delta1)
#         # Gradient of biases equal to the error
#         db2 = np.sum(delta2, axis=0, keepdims=True)
#         db1 = np.sum(delta1, axis=0, keepdims=True)
#         return dw2, db2, dw1, db1

In [297]:
def cross_entropy_loss(y_pred, y):
    return np.mean(-y * np.log(y_pred) - (1 - y) * np.log(1 - y_pred))

In [298]:
def accuracy_score(y, y_pred):
    y_pred_ = y_pred.argmax(axis=1)
    y_ = y.argmax(axis=1)
    return np.sum(y_pred_==y_) / y_.shape[0]

In [299]:
from fault_tolerant_ml.ml.ops.tensor import Tensor

In [300]:
t = Tensor(mnist.X_train)
W = Tensor(np.random.randn(784, 128))
b = Tensor(np.random.randn(1, 128))

In [301]:
x1 = Tensor(np.array(3), requires_grad=True)
x2 = Tensor(np.array(7), requires_grad=False)

interim = ((x1 + x1) + x2)
f = interim * interim

$(2x_1 + x_2)^2 = 4x_1^2 + 4x_1x_2 +x_2^2$ 

$\frac{\partial{df}}{\partial{x_1}} =8x_1 + 4x_2 = 24 + 28 = 52$

$\frac{\partial{df}}{\partial{x_2}} =4x_1 + 2x_2 = 12 + 14 = 26$

In [302]:
f.backward()

In [303]:
x1.grad

Tensor(52.0, requires_grad=False)

In [309]:
model = NeuralNet()
model.add(nn.Layer(n_inputs=784, n_outputs=128))
model.add(nn.Layer(n_inputs=128, n_outputs=10))
y_pred = model.forward(mnist.X_train)

Layer(n_inputs=784, n_outputs=128)
Layer(n_inputs=128, n_outputs=10)


In [310]:
l = 
for layer in model.layers[::-1]:
    print(layer)

Layer(n_inputs=128, n_outputs=10)
Layer(n_inputs=784, n_outputs=128)


In [9]:
# model = NeuralNet()
# print(model.fc1.shape)
# print(model.fc2.shape)
# epochs = 400
# learning_rate = 0.99
# m = mnist.X_train.shape[0]
# for epoch in np.arange(epochs):
    
#     # Feedforward
#     y_pred, z2, a1, z1 = model.forward(mnist.X_train)
    
#     # Calculate cost
#     loss = cross_entropy_loss(y_pred, mnist.y_train)
    
#     # Backprop
#     dw2, db2, dw1, db1 = model.backward(mnist.X_train, mnist.y_train, [y_pred, z2, a1, z1])
    
#     # Update weights
#     model.fc2.W = model.fc2.W - learning_rate * 1 / m * dw2
#     model.fc1.W = model.fc1.W - learning_rate * 1 / m * dw1
#     model.fc2.b = model.fc2.b - learning_rate * 1 / m * db2
#     model.fc1.b = model.fc1.b - learning_rate * 1 / m * db1
    
#     acc = accuracy_score(mnist.y_train, y_pred)
#     if epoch % 10 == 0:
#         print(f'epoch = {epoch}, loss = {loss:.3f}, TRAIN ACC = {acc:.3f}')
#     epoch += 1
    

## Autograd

## Graph

In [10]:
# g = Graph()

# g.set_as_default()

# X = Tensor(mnist.X_train)

# X + W

# W = Variable(np.random.rand(784, 128))

# b = Variable(np.zeros(shape=(1, 128)))

# z = add(matmul(X, W), b)

# z.input_nodes[0].input_nodes[0]

# z.compute()

In [11]:
from fault_tolerant_ml.ml.ops import tensor as ft

In [12]:
g = ft.Graph()
g.set_as_default()
X = ft.Tensor(mnist.X_train)
y = ft.Tensor(mnist.y_train)

In [13]:
W = ft.Tensor(np.random.randn(784, 128))

In [14]:
b = ft.Tensor(np.zeros((1, 784)))

In [15]:
a = ft.matmul(X, W)
# z = ft.add(W, b)

In [16]:
g.operations

In [17]:
def evalulate(f):
    val = []
    for i, op in enumerate(f.operations):
        print(*op.input_nodes)
        val.append(op.compute(*op.input_nodes))

In [46]:
def traverse(f):
    
    operations = []
    def recurse(node):
        if isinstance(node, ft.Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        operations.append(node)
            
    recurse(f)
    return operations

In [47]:
g = ft.Graph()
g.set_as_default()
x1 = ft.Tensor(np.array(3))
x2 = ft.Tensor(np.array(7))

In [48]:
f = ft.square(ft.add(ft.add(x1, x1), x2))

In [49]:
traverse(f)

[Tensor(3, dtype=int64),
 Tensor(3, dtype=int64),
 add(),
 Tensor(7, dtype=int64),
 add(),
 square()]

In [50]:
g.operations[0].input_nodes

[Tensor(3, dtype=int64), Tensor(3, dtype=int64)]

In [363]:
# evalulate(g)

In [362]:
# l = [ ("z1", "add", ("x1","x1")),
# ("z2", "add", ("z1","x2")),
# ("f", "square", ("z2",)) ]

# G = { "add" : lambda a,b: a+b,
# "square": lambda a:a*a }

In [360]:
# val = { "x1" : 3, "x2" : 7 }

# for step in l:
#     print(val)
#     var, op_name, func = step
#     lookup = list(map(val.get, func))
#     val[var] = G[op_name](*lookup)

In [358]:
# DG = { "add" : [ (lambda a,b: 1), (lambda a,b: 1) ],
# "square": [ lambda a:2*a ] }

# delta={}
# delta["f"] = 1
# for step in l[::-1]:
#     var, op_name, func = step
#     for op in DG[op_name]:
#         if var not in delta:
#             delta[var] = 0
#         lookup = list(map(val.get, func))
#         print(lookup)
#         delta[var] = delta[var] + DG[op_name](*lookup)

In [553]:
class Tensor(object):
    
    def __init__(self, data: np.ndarray, depends_on=None):
        
        self.depends_on = depends_on or []
        self.data = data
            
    def __add__(self, other):
        return Tensor(self.data + other.data, depends_on=[self, other])
    
    def __pow__(self, p):
        data = self.data ** p 
        return Tensor(data, depends_on=[self])
    
    def __repr__(self):
        return f"Tensor({self.data}, dtype={self.data.dtype})"


In [538]:
y1 = Tensor(np.array(3))
y2 = Tensor(np.array(7))
z1 = y1 + y1
z2 = z1 ** 2

In [539]:
z2.depends_on

[Tensor(6, dtype=int64)]

In [540]:
layers = []

In [541]:
from fault_tolerant_ml.activations.activation import Sigmoid

In [697]:
from typing import Union

In [1370]:
def ensure_array(arrayable):
    if isinstance(arrayable, np.ndarray):
        return arrayable
    else:
        return np.array(arrayable)

Tensorable = Union['Tensor', 'float', np.ndarray]

def ensure_tensor(tensorable: Tensorable):
    if isinstance(tensorable, Tensor):
        return tensorable
    else:
        return Tensor(tensorable)
    
class Tensor():
    
    def __init__(self, data, is_param=False):
        self.data = ensure_array(data)
        self._is_param = is_param
        self._grad = None
        self.shape = self.data.shape
        self.ndim = self.data.ndim
        
        if is_param:
            self.zero_grad()
        
    def __repr__(self):
        return f"Tensor({self.data}, parameter={self.is_param})"
    
    def __matmul__(self, other):
        return Tensor(self.data @ ensure_tensor(other).data)
    
    def __add__(self, other):
        return Tensor(self.data + ensure_tensor(other).data)
    
    def __radd__(self, other):
        return Tensor(self.data + ensure_tensor(other).data)
    
    def __sub__(self, other):
        return Tensor(self.data - ensure_tensor(other).data)
    
    def __rsub__(self, other):
        return Tensor(ensure_tensor(other).data - self.data)
    
    def __mul__(self, other):
        return Tensor(self.data * ensure_tensor(other).data)
    
    def __rmul__(self, other):
        return Tensor(ensure_tensor(other).data * self.data)
    
    def __truediv__(self, other):
        return Tensor(self.data / ensure_tensor(other).data)
        
    def __rtruediv__(self, other):
        return Tensor(ensure_tensor(other).data/ self.data)
    
    def __neg__(self):
        return Tensor(-self.data)
    
    def exp(self):
        return Tensor(np.exp(self.data))
    
    def log(self):
        return Tensor(np.log(self.data))
    
    def mean(self, axis=None, dtype=None, out=None):
        if dtype is None:
            dtype = self.data.dtype
        return Tensor(np.mean(self.data, axis=axis, dtype=dtype, out=out))
    
    @property
    def T(self):
        return Tensor(self.data.T)
    
    @property
    def is_param(self):
        return self._is_param
    
    @property
    def grad(self):
        return self._grad
    
    @grad.setter
    def grad(self, grad):
        self._grad = ensure_tensor(grad)
        
    def zero_grad(self):
        self.grad = Tensor(np.zeros_like(self.data))

In [1371]:
class Lay():
    
    def __init__(self, n_inputs, n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = Tensor(np.random.randn(self.n_inputs, self.n_outputs) * 0.01, is_param=True)
        self.b = Tensor(np.zeros((1, self.n_outputs)), is_param=True)
        
        self.activation_fn = Sigmoid()
        self.derived_vars = []
        
    def __call__(self, value):
        # Store input tensor for feedforward
        self.x = value
        # Store edge
        self.z = (value @ self.W) + self.b
        # Store output tensor for feedforward
        self.y = Tensor(self.activation_fn(self.z.data))
        
        return self.y
    
    def __repr__(self):
        return f"Layer({self.n_inputs}, {self.n_outputs})"

In [1372]:
class M():
    
    def __init__(self):
        self.layers = []
        
    def __repr__(self):
        return f"Model([{self.layers}])"
        
    def parameters(self):
        for layer in model.layers:
            for key, value in layer.parameters.items():
                yield value
        
    def add(self, layers):
        if isinstance(layers, list):
            self.layers += layers
        else:
            self.layers.append(layers)
            
    def zero_grad(self):
        for k, v in l1.__dict__.items():
            if isinstance(v, Tensor):
                if v.is_param:
                    v.zero_grad()
            
    def forward(self, x):
        
        input_layer = x
        for layer in self.layers:
            y_pred = layer(input_layer)
            input_layer = layer.y
            
        return y_pred

In [1373]:
class SGD():
    
    def __init__(self, loss, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.loss = loss
        
    def compute_gradients(self, model, y, y_pred):
        
        n_layers = len(model.layers)
        output_layer = model.layers[-1]
        m = model.layers[0].x.shape[0]
        # For each output unit, calculate it's error term
        delta = self.loss.grad(y, output_layer.y) * output_layer.activation_fn.grad(output_layer.z)
        output_layer.W.grad = (1 / m) * output_layer.x.T @ delta
        output_layer.b.grad = (1 / m) * np.sum(delta, axis=0, keepdims=True)

        # For hidden units, calculate error term
        for i in np.arange(n_layers - 2, -1, -1):
            delta = (delta @ model.layers[i+1].W.T) * model.layers[i].activation_fn.grad(model.layers[i].z)
            model.layers[i].W.grad = (1 / m) * (model.layers[i].x.T @ delta)
            model.layers[i].b.grad = (1 / m) * np.sum(delta, axis=0, keepdims=True)
            
    def apply_gradients(self, model):
        
        for layer in model.layers:
            layer.W = layer.W - self.learning_rate * layer.W.grad
            layer.b = layer.b - self.learning_rate * layer.b.grad

In [1374]:
from fault_tolerant_ml.losses.loss_fns import CrossEntropyLoss

In [1408]:
# Define model
model = M()
l1 = Lay(784, 128)
l2 = Lay(128, 10)
# Add layers
model.add([l1, l2])

# Tensorize numpy arrays
X = Tensor(mnist.X_train)
y = Tensor(mnist.y_train)

# Define loss
loss = CrossEntropyLoss()

# Define optimizer
optimizer = SGD(loss, learning_rate=0.99)

In [1409]:
epochs = 400
for epoch in np.arange(epochs):
    
    # Feedforward
    y_pred = model.forward(X)
    
    # Calculate loss
    epoch_loss = loss.loss(y, y_pred, reduce=True).data
    
    # Backprop
    optimizer.compute_gradients(model, y, y_pred)
    
    # Update gradients
    optimizer.apply_gradients(model)
    
    # Calculate accuracy
    acc = accuracy_score(y.data, y_pred.data)
    if epoch % 10 == 0:
        print(f"Iteration {epoch}: Loss={epoch_loss:.4f}, accuracy={acc:.4f}")
    epoch += 1

Iteration 0: Loss=0.6905, accuracy=0.0986
Iteration 10: Loss=0.3438, accuracy=0.1124
Iteration 20: Loss=0.3419, accuracy=0.1124
Iteration 30: Loss=0.3417, accuracy=0.1124
Iteration 40: Loss=0.3414, accuracy=0.1124
Iteration 50: Loss=0.3412, accuracy=0.1124
Iteration 60: Loss=0.3409, accuracy=0.1124
Iteration 70: Loss=0.3405, accuracy=0.1173
Iteration 80: Loss=0.3400, accuracy=0.1383
Iteration 90: Loss=0.3394, accuracy=0.1661
Iteration 100: Loss=0.3386, accuracy=0.1874
Iteration 110: Loss=0.3376, accuracy=0.2009
Iteration 120: Loss=0.3364, accuracy=0.2101
Iteration 130: Loss=0.3347, accuracy=0.2162
Iteration 140: Loss=0.3327, accuracy=0.2211
Iteration 150: Loss=0.3300, accuracy=0.2252
Iteration 160: Loss=0.3266, accuracy=0.2305
Iteration 170: Loss=0.3223, accuracy=0.2405
Iteration 180: Loss=0.3171, accuracy=0.2544
Iteration 190: Loss=0.3111, accuracy=0.2761
Iteration 200: Loss=0.3047, accuracy=0.3206
Iteration 210: Loss=0.2983, accuracy=0.3749
Iteration 220: Loss=0.2919, accuracy=0.4194